In [73]:

def create_equity_curve(df,z0,z1,pos):
    ec=df.diff(axis=0)
    ec.iloc[0]=0
    
    i=1
    while i<len(ec):
        ec.iloc[i]=ec.iloc[i-1]+ec.iloc[i]*z0*z1*pos
        i+=1
    return ec
    
    

def pos():
    pos=1
    while True:
        a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
        if a=='y':
            break
        if a=='n':
            pos=pos*(-1)
            break
        else:
            print "Please input either 'y' or 'n' for position"
    return pos

def z_factor(product1,product2):
    z=1
    y=1
    factor=1000
    if product1=='RB' and product2 =='RB':
        z=42
        y=42
    elif product1=='HO' and product2 =='HO':
        z=42
        y=42
    elif product1=='RB' and product2 =='HO':
        z=42
        y=42
    elif product1=='HO' and product2 =='RB':
        z=42
        y=42
    elif product1=='G' and product2 =='G':
        factor=100
    elif product1=='G' and product2 =='B':
        z=(1/7.45)
    elif product=="HO" and product2=='G':
        z=42
        y=(1/7.45)
    
    else:
        z=1
        y=1
        factor=1000
    
    return z,y, factor
        
def create_sprd(dataframe1, dataframe2):
   
    dataframe1.reset_index(inplace=True)
    dataframe2.reset_index(inplace=True)
    date=dataframe1['Date']
    
    dataframe1.drop(columns=['Date'],inplace=True)
    dataframe2.drop(columns=['Date'],inplace=True)
    
    while True:
        sprd_type=int(raw_input('Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: '))
        
        if sprd_type==1:
            
            new_df=dataframe1*42-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        elif sprd_type==2:
            
            new_df= dataframe1-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        else:
            print 'You have inputted incorrectly. Please try again and enter either integer 1 or 2.'
    

class Mth_1(object):
    
    def __init__(self):
        pass
        
    def pdt_type1(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no1(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self,start_mth,start_day,end_mth,end_day,i,j):

        count=0
        path = os.getcwd()
        
        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only

    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #don't need this, i believe
        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change', 'Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
            
            

class Mth_2(object):
    
    def __init__(self):
        pass
        
    def pdt_type2(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no2(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self, start_mth, start_day, end_mth,end_day,i,j):

        count=0
        path = os.getcwd()

        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string
     
            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only
    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change','Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
    

        
    
import numpy as np
import pandas as pd
import os
import statsmodels.tsa.stattools as ts

%matplotlib inline

#user specify instrument and contract month

product1=''
month1=''
product2=''
month2=''

front_mth=Mth_1()
product1 = front_mth.pdt_type1()
month1=front_mth.mth_no1()

back_mth=Mth_2()
product2=back_mth.pdt_type2()
month2=back_mth.mth_no2()

        

Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 4
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 7
Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 3
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 7


In [74]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [75]:
cd $product1

C:\Users\boonsin.teo\Documents\Quandl\RB


In [76]:
cd $month1

C:\Users\boonsin.teo\Documents\Quandl\RB\Jul


In [77]:
start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')
i=int(raw_input('Please input 1 if start mth is the same as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')
            
j=int(raw_input('Please input 1 if end mth is the same as contract year and 0 if not: '))


front_month=front_mth.dataframe_create(start_mth,start_day,end_mth,end_day,i,j)

Please input start mth of seasonal period in MM format: 07
Please input start day of seasonal period in DD format: 01
Please input 1 if start mth is the same as contract year and 0 if not: 0
Please input end mth of seasonal period in MM format: 06
Please input end day of seasonal period in DD format: 25
Please input 1 if end mth is the same as contract year and 0 if not: 1


In [78]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\RB


In [79]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [80]:
cd ..

C:\Users\boonsin.teo\Documents


In [81]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [82]:
cd $product2

C:\Users\boonsin.teo\Documents\Quandl\HO


In [83]:
cd $month2

C:\Users\boonsin.teo\Documents\Quandl\HO\Jul


In [84]:
back_month=back_mth.dataframe_create(start_mth, start_day,end_mth,end_day,i,j)

In [85]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\HO


In [86]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [87]:
cd ..

C:\Users\boonsin.teo\Documents


In [88]:
front_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
07/01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1128,...,3.6339,1.9999,2.0700,2.8704,NaN,2.6881,2.8454,1.889,1.6369,NaN
07/02,NaN,NaN,NaN,NaN,0.582,NaN,NaN,0.7811,NaN,1.1069,...,3.6814,1.9353,2.0479,NaN,2.4626,2.7172,2.8320,1.889,NaN,NaN
07/03,NaN,NaN,0.5817,NaN,NaN,NaN,0.7625,0.7780,NaN,NaN,...,3.7175,NaN,NaN,NaN,2.5356,2.7452,2.8231,NaN,NaN,1.6055
07/04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07/05,NaN,NaN,NaN,NaN,NaN,0.768,0.7679,NaN,NaN,NaN,...,NaN,NaN,NaN,2.8842,2.5342,2.7768,NaN,NaN,1.5795,1.5705


In [89]:
back_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
07/01,NaN,0.4993,0.5416,0.4679,0.4912,NaN,NaN,0.6540,0.6997,0.9435,...,3.9945,2.0201,2.1050,2.9696,NaN,2.8386,2.9410,1.9269,1.6215,NaN
07/02,NaN,0.4923,0.5432,0.4651,0.4947,NaN,0.6683,0.6580,0.6973,0.9343,...,4.1090,1.9664,2.0802,NaN,2.7058,2.8619,2.9229,1.9264,NaN,NaN
07/03,NaN,0.4919,0.5291,NaN,NaN,NaN,0.6715,0.6568,0.6890,NaN,...,4.1325,NaN,NaN,NaN,2.7929,2.8935,2.9127,NaN,NaN,1.5642
07/04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6159,NaN
07/05,0.4726,NaN,NaN,NaN,NaN,0.65,0.6805,NaN,NaN,NaN,...,NaN,NaN,NaN,2.9994,2.7864,2.9147,NaN,NaN,1.5724,1.5262


In [90]:
Spread_Combi=create_sprd(front_month, back_month)

Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: 2


In [91]:
Spread_Combi.dropna(axis=0,how='all',inplace=True) # drop rows with all NAs

In [92]:
Spread_Combi.dropna(axis=1,how='all',inplace=True)# drop columns with all NAs

In [93]:
adf=pd.Series()

In [94]:
for column in Spread_Combi:
    adf[column]=ts.adfuller(Spread_Combi[column])[0]


In [95]:
adf

1996   -1.315036
1997   -3.099024
1998   -1.654843
1999   -1.209066
2000    2.601595
2001   -1.078250
2002   -1.359954
2003   -2.246149
2004   -1.549876
2005   -0.606901
2006   -0.801295
2007   -1.182407
2008   -0.204269
2009   -0.737077
2010   -2.894935
2011   -3.132395
2012   -0.400991
2013   -1.354712
2014    0.245171
2015    0.713963
2016   -1.603445
2017   -2.577577
2018   -1.202230
dtype: float64

In [96]:
adf_test_stat=-2.86
adf_new=adf<adf_test_stat

In [97]:
#if reject null hypothesis = True:
adf_new

1996    False
1997     True
1998    False
1999    False
2000    False
2001    False
2002    False
2003    False
2004    False
2005    False
2006    False
2007    False
2008    False
2009    False
2010     True
2011     True
2012    False
2013    False
2014    False
2015    False
2016    False
2017    False
2018    False
dtype: bool